# 0.0 Create Azure resources

## 0.0.1 Create Azure ML workspace with Azure portal

If you don't have Microsoft Azure resource, please create it from from [this page](https://azure.microsoft.com/en-us/free/).

Once the resource is prepared, create Azure ML Workspace with the [following instruction](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-workspace?tabs=azure-portal). During the instruction, please note the following variables, which will be used in actual scenario:

- `subscription`
- `resource group`
- `region`
- `workspace name`
- `storage account`

## 0.0.2 Authentication

We adopted `CLI` and `managed identity` for AML workspace authentication. Especially, managed identity is used for computer cluster in Azure ML. Visit the site, if you're interested in authentication mechanism in Azure ML.

# 0.1 Set variables in local `.env` file

We noted plural variables like subscription in the previous section, and we put them into `.env` file for preparing our succeeding process. Please put the following format and locate it in top directory as local `.env` file.

```sh
.env
SUBSCRIPTION_ID=AAA
RESOURCE_GROUP=BBB
REGION=CCC
AML_TENANT_ID=DDD
AML_MIN_NODES=EEE
AML_MAX_NODES=FFF
  :
```

Note. `AAA`, `BBB`, .. are dummy values, and please modify them with your values. Necessary variables are as follows:

| variables in .env | description                                          |
--------------------|------------------------------------------------------- 
| SUBSCRIPTION_ID   | Subscription ID related to Azure account             |
| RESOURCE_GROUP    | Resource group name                                  |
| REGION            | Region of Azure resources                            |
| AML_TENANT_ID     | Tenant ID of Azure account                           |
| AML_MIN_NODES     | Minimum node number in submitting experiments in AML |
| AML_MAX_NODES     | Maximum node number in submitting experiments in AML |
| MODEL_INPUT_PATH  | Path of generated models                             |

# 0.2 PYthon environment

## 0.2.1 Azure ML Compute
Please create [Azure ML Compute instance](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-instance#create) with your favorite name, where we execute this script.

## 0.2.2 Library install
Please install necessary libraries as follows:

In [1]:
!pip install -r ../notebook/requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '../notebook/requirements.txt'


# 0.3 Confirm our environment variables

Please check your environment variables with the following cell.

In [6]:
!pip install python-dotenv

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv('../.env'))

ws_name = os.environ['AML_WORKSPACE_NAME']
subscription_id = os.environ['AML_SUBSCRIPTION_ID']
resource_group = os.environ['AML_RESOURCE_GROUP']
tenant_id = os.environ['AML_TENANT_ID']
min_nodes = int(os.environ['AML_MIN_NODES'])
max_nodes = int(os.environ['AML_MAX_NODES'])

aml_compute_target = os.environ['AML_COMPUTE_NAME']

print('---- Check Azure setting ----')
print(f'AML Workspace name       : {ws_name}')
print(f'Subscription ID          : {subscription_id}')
print(f'Resource group           : {resource_group}')
print(f'tenant id                : {tenant_id}')
print(f'min nodes of AML compute : {min_nodes}')
print(f'max nodes of AML compute : {max_nodes}')
print(f'AML compute target       : {aml_compute_target}')


---- Check Azure setting ----
AML Workspace name       : amlrouteoptimization
Subscription ID          : 0b3f04a9-6375-4341-a513-dd53731a99a4
Resource group           : dstoolkit-route-optimization
tenant id                : 72f988bf-86f1-41af-91ab-2d7cd011db47
min nodes of AML compute : 0
max nodes of AML compute : 4
AML compute target       : aml-compute-cluster


## 0.4 Azure ML Configuration

In [2]:
#!az login
!az login --use-device-code

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code PEM84T8FU to authenticate.
Failed to authenticate '{'additional_properties': {}, 'id': '/tenants/16b3c013-d300-468d-ac64-7eda0820b6d3', 'tenant_id': '16b3c013-d300-468d-ac64-7eda0820b6d3', 'tenant_category': 'Home', 'country': None, 'country_code': 'US', 'display_name': 'Microsoft Non-Production', 'domains': ['fdpo.onmicrosoft.com'], 'tenant_id_name': "16b3c013-d300-468d-ac64-7eda0820b6d3 'Microsoft Non-Production'"}' due to error 'AADSTS50005: User tried to log in to a device from a platform (Unknown) that's currently not supported through Conditional Access policy. Supported device platforms are: iOS, Android, Mac, and Windows flavors.
Trace ID: a466e119-b7a1-45d7-95ac-38c0de521300
Correlation ID: c444ae34-814d-47d0-b15b-67f72ab4673a
Timestamp: 2022-08-31 09:11:59Z'
The following tenants don't contain accessible subscriptions. Use 'az login --allow-no-subscriptions' to have tenant leve

In [5]:
from azureml.core.authentication import AzureCliAuthentication
from azureml.core import Workspace

cli_auth = AzureCliAuthentication()
ws =  Workspace.get(name=ws_name
                    ,subscription_id=subscription_id
                    ,resource_group=resource_group
                    ,auth=cli_auth)